## Building the Predictor

### Setup

In [23]:
import sys
import os
import time

import boto3

#importing forecast utility from common directory
sys.path.insert(0, os.path.abspath("common"))
import util

In [24]:
%store -r

In [25]:
session = boto3.Session(region_name = region)
forecast = session.client(service_name = "forecast")
forecastquery = session.client(service_name = 'forecastquery')

## Create a Predictor

In [26]:
predictor_name = project + '_deeparp_algo'

In [27]:
forecastHorizon = 24

In [28]:
algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

In [29]:
create_predictor_response=forecast.create_predictor(PredictorName=predictor_name, 
                                                  AlgorithmArn=algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                         "BackTestWindowOffset": 24}, 
                                                  InputDataConfig= {"DatasetGroupArn": datasetGroupArn},
                                                  FeaturizationConfig= {"ForecastFrequency": "H", 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreatePredictor operation: A predictor already exists with the arn: arn:aws:forecast:us-east-1:879335107588:predictor/active_power_forecast_deeparp_algo

In [ ]:
predictor_arn = create_predictor_response['PredictorArn']

In [31]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_predictor(PredictorArn=predictor_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

ACTIVE 


## Get Error Metrics

In [30]:
forecast.get_accuracy_metrics(PredictorArn = predictor_arn)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
   'TestWindows': [{'EvaluationType': 'SUMMARY',
     'Metrics': {'RMSE': 54.93052509695769,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.46805555759966944},
       {'Quantile': 0.5, 'LossValue': 0.6736751592853709},
       {'Quantile': 0.1, 'LossValue': 0.17039252357842907}]}},
    {'TestWindowStart': datetime.datetime(2010, 10, 31, 6, 44, tzinfo=tzlocal()),
     'TestWindowEnd': datetime.datetime(2010, 11, 1, 6, 44, tzinfo=tzlocal()),
     'ItemCount': 1,
     'EvaluationType': 'COMPUTED',
     'Metrics': {'RMSE': 54.93052509695769,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.46805555759966944},
       {'Quantile': 0.5, 'LossValue': 0.6736751592853709},
       {'Quantile': 0.1, 'LossValue': 0.17039252357842907}]}}]}],
 'ResponseMetadata': {'RequestId': 'f54bc688-9638-41db-ad5b-92404b887d2f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': 

## Create a forecast 

In [18]:
forecastName = project + '_deeparp_algo_forecast'

In [19]:
create_forecast_response = forecast.create_forecast(ForecastName = forecastName,
                                                   PredictorArn = predictor_arn)
forecastArn = create_forecast_response['ForecastArn']

In [33]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_forecast(ForecastArn=forecastArn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

ACTIVE 


## Get Forecast

In [34]:
print(forecastArn)
print()
forecastResponse = forecastquery.query_forecast(
    ForecastArn=forecastArn,
    Filters={"item_id":"client_1"}
)
print(forecastResponse)

arn:aws:forecast:us-east-1:879335107588:forecast/active_power_forecast_deeparp_algo_forecast

{'Forecast': {'Predictions': {'p10': [{'Timestamp': '2010-11-01T00:00:00', 'Value': 46.170188903808594}, {'Timestamp': '2010-11-01T01:00:00', 'Value': 16.540122985839844}, {'Timestamp': '2010-11-01T02:00:00', 'Value': 12.155609130859375}, {'Timestamp': '2010-11-01T03:00:00', 'Value': 16.15972900390625}, {'Timestamp': '2010-11-01T04:00:00', 'Value': 15.278203964233398}, {'Timestamp': '2010-11-01T05:00:00', 'Value': 17.611433029174805}, {'Timestamp': '2010-11-01T06:00:00', 'Value': 21.751605987548828}, {'Timestamp': '2010-11-01T07:00:00', 'Value': 40.40333557128906}, {'Timestamp': '2010-11-01T08:00:00', 'Value': 28.807479858398438}, {'Timestamp': '2010-11-01T09:00:00', 'Value': 31.1025390625}, {'Timestamp': '2010-11-01T10:00:00', 'Value': 19.79860496520996}, {'Timestamp': '2010-11-01T11:00:00', 'Value': 21.992919921875}, {'Timestamp': '2010-11-01T12:00:00', 'Value': 22.907835006713867}, {'Timest

## Next Steps

In [35]:
%store forecastArn
%store predictor_arn

Stored 'forecastArn' (str)
Stored 'predictor_arn' (str)
